In [95]:
import requests
import pandas as pd
import json
from time import sleep

In [96]:
def lire_csv(nom_fichier):
    '''Cette fonction permet de lire un fichier csv et de l'ouvrir dans un dataframe'''
    dataframe = pd.read_csv(nom_fichier)
    return dataframe

In [97]:
def requete_info_gare(page):
    url = f"https://ressources.data.sncf.com/api/explore/v2.1/catalog/datasets/frequentation-gares/records?limit=100&offset={page*100}"
    reponse = requests.get(url).json()
    return reponse

In [98]:
def enregistrer_json(data,nom_fichier):
    '''Cette fonction enregistre un fichier json'''
    with open(nom_fichier,"w") as fc:
        json.dump(data,fc)

In [99]:
def dataframe_to_csv(dataframe,nomfichier):
    '''Cette fonction enregistre un dataframe en csv'''
    dataframe.to_csv(nomfichier)

In [100]:
def calcul_nombre_page():
        
    premiere_requete = requete_info_gare(0)
    nb_gare = premiere_requete["total_count"]
    nb_page = nb_gare // 100
    if nb_gare % 100 != 0:
        nb_page += 1

    return nb_page

In [101]:
def recuperer_toutes_pages(nb_pages):
    '''Cette fonction fait une requete api pour chaque page et stocke les resultats dans une liste'''
    #Initie liste gare
    liste_info_gare = []
    # Requete api pour chaque page
    for page in range(nb_pages):
        #recupère les données de la clé results
        data_page = requete_info_gare(page)["results"]
        #ajoute les données à la liste, element par element
        for item in data_page:
            liste_info_gare.append(item)
    return liste_info_gare  

In [102]:
def fichier_to_dataframe(chemin: str):
    '''Cette fonction ouvre un fichier json ou csv et place son contenu dans un dataframe'''

    #J'utilise split pour connaitre l'extension de mon fichier
    extension = chemin.split(".")[-1]
    if extension == "csv":
        dataframe = pd.read_csv(chemin)
    elif extension == "json":
        dataframe = pd.read_json(chemin)
    else:
        print("L'extension du fichier doit être csv ou json")

    return dataframe

In [103]:
def clear_data(dataframe):
    '''Cette fonction selectionne deux colonne sur un dataframe, cast la colonne code en int, et ajoute un prefixe'''
    
    dataframe_deux_colonnes = dataframe[["code_uic","total_voyageurs_2022"]]
    # Je cast la colonne code_uic en string
    dataframe_deux_colonnes.loc[:,"code_uic"] = dataframe_deux_colonnes.loc[:,"code_uic"].astype("string")
    # J'applique une fonction lambda sur la colone code_uic pour ajouter un prefix à toutes les valeurs
    dataframe_deux_colonnes.loc[:,"code_uic"] = dataframe_deux_colonnes.loc[:,"code_uic"].apply(lambda x: "stop_area:SNCF:"+x)
    # Je renomme la colonne
    dataframe_deux_colonnes = dataframe_deux_colonnes.rename(columns={'code_uic':'id'})

    
    return dataframe_deux_colonnes

In [104]:
def joindre_dataframe(df1,df2):
    '''Cette fonction permet de joindre deux dataframe'''
    df3 = pd.merge(df1,df2, how='inner', on="id")
    return df3

In [105]:
def top100(df):
    '''Cette fonction classe un dataframe selon la frequentation des gare renvoie les 100 premières valeurs'''

    #Range les enregistrements en fonction du nombre de voyageur
    df_ranked = df.sort_values("total_voyageurs_2022", ascending=False)
    #Les 100 premières valeur
    df_100 = df_ranked.iloc[:100]

    return df_100   

In [106]:
def main():
    #Calcul du nombre de page
    nb_page = calcul_nombre_page()
    #Fais les requetes sur toutes les pages
    data_frequentation_brut = recuperer_toutes_pages(nb_page)
    #Enregistrement du fichier json
    enregistrer_json(data_frequentation_brut,"data/donnees_brutes_frequentation.json")
    #Creation du dataframe
    dataframe_brutes = fichier_to_dataframe("data/donnees_brutes_frequentation.json")
    #Nettoyage des données
    data_frequentation = clear_data(dataframe_brutes)
    #Recupération des information des gares
    data_gare = fichier_to_dataframe("data/liste_gare.csv")
    #Jointure des deux tables
    tables_jointes = joindre_dataframe(data_frequentation,data_gare)
    #Sauvegarde en csv
    tables_jointes.to_csv("data/liste_gare_et_frequentation.csv")
    # Classement des 100 premieres valeurs
    top_gares = top100(tables_jointes)
    #Enregistrement
    top_gares.to_csv("data/top100gare.csv")

In [107]:
main()

C:\Users\steve\AppData\Local\Temp\ipykernel_20968\1856679397.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<StringArray>
['87757559', '87781278', '87746115', '87615195', '87743716', '87743146',
 '87172858', '87286856', '87721324', '87757567',
 ...
 '87725820', '87734723', '87713511', '87184531', '87143024', '87172221',
 '87343301', '87286229', '87281436', '87182378']
Length: 3021, dtype: string' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dataframe_deux_colonnes.loc[:,"code_uic"] = dataframe_deux_colonnes.loc[:,"code_uic"].astype("string")
